In [ ]:
!pip3 install wandb

In [ ]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'chatbot'
device = 'cuda'

In [ ]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

In [ ]:
tokenize('$100')

In [ ]:
def stem(word):
    return stemmer.stem(word.lower())

In [ ]:
stem('organic')

In [ ]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [ ]:
bag_of_words(['hi'],['hi','how','hi'])

In [ ]:
# data = pd.read_csv('./data.csv').sample(frac=1)[:3250]

In [ ]:
# data.to_csv('./cleaned-data.csv',index=False)

In [ ]:
data = pd.read_json('./data.json')

In [ ]:
dict(data.iloc[0]['intents'])

In [ ]:
new_data = {'Question':[],'Tag':[]}
responses = {'Tag':[],'Response':[]}

In [ ]:
for i in range(len(data)):
    info = data.iloc[i]['intents']
    for q in list(info['patterns']):
        new_data['Tag'].append(info['tag'])
        new_data['Question'].append(q)
    for r in list(info['responses']):
        responses['Tag'].append(info['tag'])
        responses['Response'].append(r)

In [ ]:
data = pd.DataFrame(new_data)

In [ ]:
data.to_csv('./data.csv')

In [ ]:
responses = pd.DataFrame(responses)

In [ ]:
X = data['Question']
y = data['Tag']
X_words = []
labels = {}
labels_r = {}
idx = 0
data = []

In [ ]:
for label in y:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [ ]:
labels

In [ ]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    X_words.extend(new_X)
    
    data.append([
        X_words,
        np.eye(labels[y_batch],len(labels))[labels[y_batch]-1]
    ])

In [ ]:
X_words = sorted(set(X_words))
np.random.shuffle(data)

In [ ]:
X = []
y = []

In [ ]:
for input_sentence,output_sentence in tqdm(data):
    X.append(bag_of_words(input_sentence,X_words))
    y.append(output_sentence)

In [ ]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.0125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [ ]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [ ]:
def get_accuracy(model,X,y,):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [ ]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 1024*2
        self.activation = ReLU()
        self.input = Linear(len(X_words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear3 = Linear(self.hidden,self.hidden)
        self.linear4 = Linear(self.hidden,self.hidden)
        self.linear5 = Linear(self.hidden,self.hidden)
        self.linear6 = Linear(self.hidden,self.hidden)
        self.linear7 = Linear(self.hidden,self.hidden)
        self.linear8 = Linear(self.hidden,self.hidden)
        self.linear9 = Linear(self.hidden,self.hidden)
        self.linear10 = Linear(self.hidden,self.hidden)
        self.output = Linear(self.hidden,len(labels))
        
    def forward(self,X):
        preds = self.input(X)
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.activation(self.linear6(preds))
        preds = self.activation(self.linear7(preds))
        preds = self.activation(self.linear8(preds))
        preds = self.activation(self.linear9(preds))
        preds = self.activation(self.linear10(preds))
        preds = self.output(preds)
        return preds

In [ ]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 500

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    preds = model(X_train)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()

In [49]:
torch.save(model.to('cpu'),'model.pt')
torch.save(model.to('cpu'),'model.pth')
torch.save(model.to('cpu').state_dict(),'model-sd.pt')
torch.save(model.to('cpu').state_dict(),'model-sd.pth')
torch.save(X_words,'words.pt')
torch.save(X_words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')
torch.save(labels_r,'labels_r.pt')
torch.save(labels_r,'labels_r.pth')
torch.save(idx,'idx.pt')
torch.save(idx,'idx.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')
torch.save(y,'y.pt')
torch.save(y,'y.pth')

In [37]:
torch.argmax(model(torch.from_numpy(bag_of_words(['hi','how','are','you'],X_words)).to(device).float()))

tensor(0, device='cuda:0')

In [42]:
responses

Tag                                           Response
0   greeting                                            Hey :-)
1   greeting                         Hello, thanks for visiting
2   greeting                   Hi there, what can I do for you?
3   greeting                          Hi there, how can I help?
4    goodbye                 See you later, thanks for visiting
5    goodbye                                    Have a nice day
6    goodbye                         Bye! Come back again soon.
7     thanks                                     Happy to help!
8     thanks                                          Any time!
9     thanks                                        My pleasure
10     items                             We sell coffee and tea
11     items                             We have coffee and tea
12  payments              We accept VISA, Mastercard and Paypal
13  payments      We accept most major credit cards, and Paypal
14  delivery                            Delivery takes 2-4 days
15  delivery                            Shipping takes 2-4 days
16     funny  Why did the hipster burn his mouth? He drank t...
17     funny  What did the buffalo say when his son left for...

['Hey :-)',
 'Hello, thanks for visiting',
 'Hi there, what can I do for you?',
 'Hi there, how can I help?']

In [48]:
responses.to_csv('./responses.csv')